In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import glob
import time
%matplotlib inline

In [ ]:
img = cv2.imread('e:/pics/test/img_10.jpg')
img.shape

In [ ]:
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img.shape

In [ ]:
img = cv2.resize(img, (9,8))

In [ ]:
img.shape

In [ ]:
diff = img[:, 1:] > img[:, :-1]
diff

In [ ]:
r = sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])
print(r)
print(type(r))

In [ ]:
def dhash(file):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (9,8))
    diff = img[:, 1:] > img[:, :-1]
    return sum([2 ** i for (i, v) in enumerate(diff.flatten()) if v])

In [ ]:
r1 = dhash('e:/pics/test/img_10.jpg')
r2 = dhash('e:/pics/test/img10_1.jpg')
print(r1==r2)

In [ ]:
import photohash
distance = photohash.distance('e:/pics/test/img_10.jpg', 'e:/pics/test/img10_1.jpg')

In [ ]:
distance

In [ ]:
%timeit photohash.distance('e:/pics/test/img_10.jpg', 'e:/pics/test/img10_2.jpg')

In [ ]:
photohash.distance('e:/pics/test/img_10.jpg', 'e:/pics/test/img_4.jpg')

In [ ]:
photohash.average_hash('e:/pics/test/img_10.jpg')

In [ ]:
photohash.average_hash('e:/pics/test/img10_2.jpg')

In [ ]:
def hash_img(file):
    img = Image.open(file)
    img = img.resize((8,8),Image.ANTIALIAS).convert('L')
    pixels = list(img.getdata())
    avg = sum(pixels) / len(pixels)
    bits = "".join(map(lambda pixel: '1' if pixel > avg else '0', pixels))
    return bits
bits = hash_img('e:/pics/test/img_10.jpg')
bits

In [ ]:
hashformat = "0{hashlength}x".format(hashlength=8 ** 2 // 4)

In [ ]:
hashformat

In [ ]:
int(bits, 2).__format__(hashformat)

In [ ]:
img2 = hash_img('e:/pics/test/img10_1.jpg')
img2

In [ ]:
img2==bits

In [ ]:
img3 = hash_img('e:/pics/test/img_4.jpg')
img3

In [ ]:
img.size
#gray

In [ ]:
np.array(img)==np.array(img2)

In [13]:
def average_hash(image_path, hash_size=8):
    """ Compute the average hash of the given image. """
    with open(image_path, 'rb') as f:
        # Open the image, resize it and convert it to black & white.
        image = Image.open(f).resize((hash_size, hash_size), Image.ANTIALIAS).convert('L')
        pixels = list(image.getdata())

    avg = sum(pixels) / len(pixels)

    # Compute the hash based on each pixels value compared to the average.
    bits = "".join(map(lambda pixel: '1' if pixel > avg else '0', pixels))
    hashformat = "0{hashlength}x".format(hashlength=hash_size ** 2 // 4)
    return int(bits, 2).__format__(hashformat)

# compare bit each by each
def distance(left_hash, right_hash):
    if len(left_hash)!=len(right_hash):
        raise ValueError('hamming distance not match length')
    return sum(map(lambda x: 0 if x[0] == x[1] else 1, zip(left_hash, right_hash)))

def to_numpy(s):
    return np.array([x for x in s])

In [ ]:
for i,f in enumerate(glob.glob('e:/pics/mag/*.jpg')):
    print(f)
    break

In [ ]:
features = []
for i,f in enumerate(glob.glob('e:/pics/mag/*.jpg')):
    try:
        tic = time.time()
        h = average_hash(f)
        h = to_numpy(h)
        toc = time.time()
        if i%100==0:
            print('ok', len(features), (toc-tic))
        features.append(h)
    except Exception as e:
        print('fail', f, e)
#     break
    


In [ ]:
features = np.array(features)
print(features.shape)

In [ ]:
a = average_hash('e:/pics/test/img_4.jpg')
b = average_hash('e:/pics/test/img_10.jpg')
print(a, b)
print(distance(a, b))

In [ ]:
a = average_hash('e:/pics/test/img10_1.jpg')
b = average_hash('e:/pics/test/img_10.jpg')
c = average_hash('e:/pics/test/img_4.jpg')
print(a, b)
print(distance(a, b))

In [ ]:
%timeit distance(a, b)
# 3.06us for two compare, but what for n? nx3us?


In [ ]:
b = np.array([x for x in b])
c = np.array([x for x in c])
features = np.array([b,c])
print(features.shape)

In [ ]:
a = np.array([x for x in a])

In [ ]:
a = average_hash('e:/pics/test/img_308.jpg')
a = to_numpy(a)
print(features.shape)
a = a.reshape(1,-1)
print(a.shape)
bindiff = a!=features
hammingdiff = bindiff.sum(axis=1)
rank = np.argsort(hammingdiff)
print('min', np.min(hammingdiff))
print(rank)

In [ ]:
np.sort(hammingdiff)[:10]

In [ ]:
np.min(hammingdiff)

In [ ]:
np.argmin(hammingdiff.reshape(1,-1), axis=1)

In [ ]:
hammingdiff.shape

In [5]:
a = np.array([10,11,5,0,1,9])
np.argsort(a[a<=1])

array([0, 1], dtype=int64)

In [11]:
r = np.where(a<=1)[0]

In [12]:
r.size

2

In [10]:
r.tolist()

[3, 4]

In [25]:
a = average_hash('d:/tmp/a.jpg')
b = average_hash('d:/tmp/a1.jpg')

In [26]:
a = to_numpy(a)
b = to_numpy(b)

In [27]:
f = np.array([a,b,a])

In [29]:
b = b.reshape(1,-1)
b.shape

(1, 16)

In [39]:
ids = np.array(['a','b','a'])

In [31]:
s = (f!=b).sum(axis=1)

In [35]:
index = np.where(s<=1)[0]

In [40]:
ids[index]

array(['a', 'b', 'a'], dtype='<U1')

In [41]:
np.__version__

'1.14.0'